In [21]:
from PIL import Image
from src.color import Color
from rembg import remove
from os import listdir
from os.path import isfile, join
# from octree_quantizer import OctreeQuantizer

In [19]:
REMOVE_BACKGROUND = False

## Remove background from images

In [34]:
# Remove backgroud from image
if (REMOVE_BACKGROUND):
    input_path = "./dataset/bg/"
    output_path = "./dataset/no_bg/"

    files = [f for f in listdir(input_path) if isfile(join(input_path, f))]

    for idx,img in enumerate(files):
        input = Image.open(join(input_path, img))
        output = remove(input)
        output.save(join(output_path, str(idx) + ".png"))


## Load image to octree

In [14]:
image = Image.open('./dataset/no_bg/1.png')
pixels = image.load()
width, height = image.size

# add colors to the octree
for j in range(height):
    for i in range(width):
        Color(*pixels[i, j])
        ruuf = 0